In [2]:
!pip install firebase-admin
!pip install google-cloud-storage
!pip install python-dotenv
!pip install pandas


   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
    --------------------------------------- 0.3/13.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.2 MB 882.6 kB/s eta 0:00:15
   - -------------------------------------- 0.5/13.2 MB 882.6 kB/s eta 0:00:15
   -- ------------------------------------- 0.8/13.2 MB 958.5 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/13.2 MB 1.0 MB/s eta 0:00:12
   --- ------------------------------------ 1.3/13.2 MB 1.1 MB/s eta 0:00:11
   ----- ---------------------------------- 1.8/13.2 MB 1.2 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/13.2 MB 1.2 MB/s eta 0:00:10
   ------- -------------------------------- 2.4/13.2 MB 1.3 MB/s eta 0:00:09
   -------- ------------------------------- 2.9/13.2 MB 1.4 MB/s eta 0:00:08
   ---------- ----------------------------- 3.4/13.2 MB 1.4 MB/s eta 0:00:07
   -----------

In [3]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

In [4]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  



In [5]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffe-shop-chat-bot.firebasestorage.app',
     'databaseURL': 'https://coffe-shop-chat-bot-default-rtdb.firebaseio.com'
})

In [6]:
bucket = storage.bucket()

# Upload Data

In [7]:
image_folder_path = './products/images/'

In [8]:
products_collection = db.reference('products')

In [9]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [10]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [11]:
for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path,row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)
    

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup
